In [1]:
import pandas as pd
import json
import ollama
from langchain.llms import Ollama

In [2]:
df = pd.read_csv('../Training Dataset/FewShotForEvents.csv')
dfReviews = df['content'].tolist()
dfReviews

["Keeps logging me out of account saying session expired and everytime I try to log in it won't let me, keeps updating constantly even when I keep it up to date so not sure if the app is glitching, overall not a good app so may have to consider deleting Facebook as this has put me off and happens way too much, keep trying to update the app thinking it was that but its still not working and it says update again even after i already did",
 'Facebook marketplace has a bug or a terrible feature where it will select a random picture as the main one (so the only one buyers see when searching and scrolling) keeps selecting a zoomed up tiny area of damage as the main photo instead of the actual overall photo of an object no wonder no one but scam bots message me now.',
 "I don't know what has happened lately it will not let me log into my Facebook I keep logging in right password and all cuz it starts to log in but then it decides login has expired and kicks me right back out logs me out then 

In [3]:
ollama = Ollama(base_url='http://localhost:11434', model='EventExtraction-llama2')

In [4]:
def predict_event(reviews):
    events = []
    for review in reviews:
        event = ollama(review)
        events.append(event)
    return events

In [5]:
zeroShotEvents = predict_event(dfReviews)

d:\Academics\Thesis\New folder\Web-Scraping\venv\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AttributeError: 'list' object has no attribute 'head'

In [16]:
zeroShotEvents

['\n[User Action Events]\n\n* Logging in to the app\n* Updating the app\n* Trying to log in after updating the app\n\n[App Problem Events]\n\n* Sessions expiring unexpectedly\n* App glitching or freezing\n* Inability to log in despite updates\n\nNote: These events are organized based on their relationships and similarities, but they can be further refined and grouped into more specific categories depending on the context and purpose of the analysis.',
 "\n[User Action Events]\n\n* User searches for an object on Facebook Marketplace\n* User scrolls through the search results\n* App selects a random picture as the main image (only one buyers see when searching and scrolling)\n* User tries to view the full photo of the object, but the app selects a zoomed-up tiny area of damage instead of the actual overall photo.\n\n[App Problem Events]\n\n* The app violates user expectations by selecting an incorrect main image for search results\n* The app does not display the full photo of the object 

In [17]:
def extract_events(event_text):
    user_actions = ""
    app_problems = ""
    parts = event_text.split("[App Problem Events]")
    if len(parts) > 1:
        user_actions = parts[0].replace("[User Action Events]", "").strip()
        app_problems = parts[1].strip()
    return user_actions, app_problems

In [18]:
extracted_events = [extract_events(event) for event in zeroShotEvents]
zeroShotEventPairs = pd.DataFrame(extracted_events, columns=["User Action Events", "App Problem Events"])
zeroShotEventPairs.head()

,User Action Events,App Problem Events
0,* Logging in to the app\n* Updating the app\n*...,* Sessions expiring unexpectedly\n* App glitch...
1,* User searches for an object on Facebook Mark...,* The app violates user expectations by select...
2,,
3,,
4,,


In [19]:
zeroShotEventPairs.to_csv('../Generated Dataset/Zero Shot/LLAMA2/ZeroShotEventPairs.csv', index=False)

                                    User Action Events  \
0    * Logging in to the app\n* Updating the app\n*...   
1    * User searches for an object on Facebook Mark...   
2                                                        
3                                                        
4                                                        
..                                                 ...   
325  * Making an audio or video call\n* Data being ...   
326  * Interacting with the app (using WhatsApp)\n*...   
327                                                      
328                                                      
329                                                      

                                    App Problem Events  
0    * Sessions expiring unexpectedly\n* App glitch...  
1    * The app violates user expectations by select...  
2                                                       
3                                                       
4                 